# High-fidelity model of lungs connected to mechanical ventilation: simulations and dataset creation.

In the following notebook we describe the methodology to simulate a high-fidelity lung model assisted by pressure-controlled mechanical ventilation, using the approach proposal in [Avilés & Hurtado (2022)](https://www.frontiersin.org/articles/10.3389/fphys.2022.984286/full).


---



Regarding the formulation, the lung is represented by a poroelastic continuum $Ω_0$ with boundary $Γ_0$. The boundary of the lung domain was partitioned into the airways surface and the
visceral pleura surface that lines the remaining lung surface. The airways boundary was
determined by considering the surface encompassing bifurcations from the mediastinal sur-
face down to the lobar bronchi. Smaller airways in subsequent branches were considered
to be part of the lung parenchyma domain. The visceral pleura surface was defined as the
complement of the airways surface.

The strong formulation for the finite strain poroelasticity problem is

$$\text{Find $\vec{\varphi} \in C^2(\Omega_0 \times [0,T],R^N)$ and $P_{alv} \in C^2(\Omega_0 \times [0,T],R)$ such as:}   \nonumber$$

\begin{align}
\text{Div} (\boldsymbol{P})+R\vec{B} &=\boldsymbol{0} \quad & \text{in } \Omega_0 \times (0,T].\\
 \frac{\partial \Phi}{\partial t} + \text{Div} (\boldsymbol{Q}) &= 0  \quad & \text{in } \Omega_0 \times (0,T]. \\
\boldsymbol{\varphi} &= \boldsymbol{\varphi}_0  \quad &  \text{in } \Omega_0.\\
P_{alv} &= P_0  & \text{in } \Omega_0.\\
\boldsymbol{\varphi} &= \boldsymbol{\vec{\varphi}} \quad & \text{on } \Gamma_{\varphi} \times (0,T].\\
\boldsymbol{P}\cdot \boldsymbol{N} &= \bar{\boldsymbol{T}} \quad & \text{on } \Gamma_T \times (0,T]. \\
{P_{alv}} &= \bar{{P}} \quad & \text{on } \Gamma_P \times (0,T].\\
\boldsymbol{Q}\cdot \boldsymbol{N} &= \bar{{Q}} \quad & \text{on } \Gamma_Q \times (0,T].
\end{align}

We begin by importing the necessary modules.


---


\
**Note**: Up to this point, it is recommended to upload to this session the `modelfunctions` and `linearregression` functions to be imported, along with `mesh.h5` file (mesh of the high-fidelity lung model).

In [ ]:
# Import and install (***it is necessary to run this cell 2 times!***)
try:
    import dolfin
    print("oui")
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
!pip install meshio==4.4.6

In [ ]:
!pip install pyDOE

In [ ]:
import meshio
import dolfin
import os
import matplotlib.pyplot as plt
import numpy as np
import time
import modelfunctions
import linearregression

from ast import Interactive
from dolfin import *
from modelfunctions import solve_poroelasticity
from linearregression import regression
from sklearn import linear_model
from pyDOE import lhs

## Parameters dataset creation

1. We specify a `range` around the baseline values of the considered lung parameter values under analysis:

Lung tissue constitutive model parameters:
*   $c$
*   $\beta$
*   $c_1$
*   $c_3$

Lung permeability:
*   $k$

Spring stiffness (chest-wall effect):
*   $K_s$


\
2. We use Latin Hypercube Sampling to obtain `sample_num` number of samples of the parameter space, bounded in the specified range.


---


\
**Note**: Just for demonstrative purposes, here we only obtain 2 samples and a range of 0% (this in order to check that the sampling is correctly performed, and to obtain the response of our high-fidelity model when using the baseline values for the parameters).

In [ ]:
# We consider a range of 50% around the mean
range = 50/100

# Constitutive model parameters bounds (Bir. 2019 et al.)
C_bir2019    = [-range*356.7 + 356.7 , 356.7 +range*356.7] # parameter c
Beta_bir2019 = [-range*1.075 + 1.075 , 1.075 +range*1.075] # parameter beta
C1_bir2019   = [-range*278.2 + 278.2 , 278.2 +range*278.2] # parameter c1
C3_bir2019   = [-range*5.766 + 5.766 , 5.766 +range*5.766] # parameter c3

# Mechanical parameters bounds
per = [-range*10000 + 10000 , 10000 +range*10000]          # parameter k
KKresortee = [-range*0.08 + 0.08 , 0.08 +range*0.08]       # parameter Ks

# We use Latin Hypercube Sampling approach to generate space-filling training samples.
# Number of samples
sample_num = 2
# Bounds
lb = np.array([C_bir2019[0], Beta_bir2019[0], C1_bir2019[0], C3_bir2019[0], per[0], KKresortee[0]])
ub = np.array([C_bir2019[1], Beta_bir2019[1], C1_bir2019[1], C3_bir2019[1], per[1], KKresortee[1]])
# Generation of samples
X_data = (ub-lb)*lhs(6, samples=sample_num) + lb

# We create an output vector to store the posterior simulation results (Crs and R)
Y_data = np.zeros([sample_num, 2])
print(X_data, Y_data)

[[3.567e+02 1.075e+00 2.782e+02 5.766e+00 1.000e+04 8.000e-02]
 [3.567e+02 1.075e+00 2.782e+02 5.766e+00 1.000e+04 8.000e-02]] [[0. 0.]
 [0. 0.]]


## Performing the simulations

Next, for each sample of model parameters, the problem is solved using finite elements in FEniCS. To this end we call `solve_poroelasticity`, in which the following steps are performed:

*   Load the mesh and boundaryes
*   Build function space. We use Taylor-Hood element
*   Initialize solver
*   Time-stepping loop

At the end of each simulation, we obtain arrays of pressure, airflow, and volume waveforms, along with a time array.




In [ ]:
# Measuring time
start_time = time.time()

# Simulation loop (it will execute 'sample_num' number of cases)
i = 0

while i < len(X_data):

    ii = i
    # Obtain the model parameters of the 'i' sample
    C_bir2019    = X_data[i,0]
    Beta_bir2019 = X_data[i,1]
    C1_bir2019   = X_data[i,2]
    C3_bir2019   = X_data[i,3]
    per          = X_data[i,4]
    KKresortee   = X_data[i,5]

    models=['bir2019']
    # We use these parameters in our simulation, via 'solve_poroelasticity'
    for model in models:
        tiempos,Jacob,flux,presionestodas=solve_poroelasticity('TEST',model,'high',per,KKresortee,ii,C_bir2019,Beta_bir2019,C1_bir2019,C3_bir2019)

    i += 1

print("--- %s seconds ---" % (time.time() - start_time))

## Obtaining respiratory-system compliance and airways resistance

Finally, from the ventilator signals and using the single compartment equation of motion, we obtain the respiratory-system compliance and airways resistance parameters via least-squares fitting by calling the `regression` function:

\begin{equation}
\text{P}_{\text{aw}}(t)=\frac{\text{V}(t)}{\text{C}_{\text{rs}}}+ \text{R} \dot{\text{V}}(t)+\text{PEEP}-\text{P}_{\text{mus}}(t)
\end{equation}

In [ ]:
# Simulation loop (it will execute 'sample_num' number of cases)
i = 0

while i < len(X_data):

    ii = i

    name='bir2019'

    ## We load the mechanical ventilation curves (considering both lungs)
    fluxes=fileflujos=np.load(name+str(ii)+'fluxes.npy')+np.load(name+str(ii)+'fluxes.npy')#[0:84]
    presionestodas=filepresiones=np.load(name+str(ii)+'presionestodas.npy')#+filepresionesi
    tiempos=filetiempos=np.load(name+str(ii)+'tiempos.npy')
    Jacob=filevolumenes=np.load(name+str(ii)+'volumenes.npy')+np.load(name+str(ii)+'volumenes.npy')#[0:84]
    Jacob=Jacob-Jacob[0]

    # We read the generated .npy files from both lungs
    maflujos=fluxes*60
    mapresiones=presionestodas
    matiempos=tiempos
    mavolumenes=Jacob

    # We add a 0 in the first element of the arrays (due to missing data from simulations)
    maflujos=np.concatenate((np.array([0]),np.asarray(maflujos)))
    mapresiones=np.concatenate((np.array([0]),np.asarray(mapresiones)))
    matiempos=np.concatenate((np.array([0]),np.asarray(matiempos)))
    mavolumenes=np.concatenate((np.array([0]),np.asarray(mavolumenes)))

    # We adjust the respiratory-system compliance and airways resistance from the equation of motion
    regression(maflujos,mapresiones,matiempos,mavolumenes,name,Y_data,i)

    i += 1

bir2019
Resistance(R)= (3.8195726868217355, 2) cm H2O L/S, 
Compliance (Crs)= 93.62131911065362 ml/cm H2O
---------------
bir2019
Resistance(R)= (3.8195726868217355, 2) cm H2O L/S, 
Compliance (Crs)= 93.62131911065362 ml/cm H2O
---------------


## Display the dataset

Finally, we display the created dataset. From this, it can be observed that:

- `X_data` is an array that corresponds to the sampled lung parameters using the Latin hypercube design: each row a sample containing the values of the six parameters.

- `Y_data` is an array where a single row corresponds to the obtained respiratory-system compliance and airways resistance after simulating the corresponding sample of lung model parameters.

In [ ]:
# Print input and output arrays
print(X_data)
print(Y_data)

# Saving arrays
np.save('input_data.npy', X_data)
np.save('output_data.npy', Y_data)

[[3.567e+02 1.075e+00 2.782e+02 5.766e+00 1.000e+04 8.000e-02]
 [3.567e+02 1.075e+00 2.782e+02 5.766e+00 1.000e+04 8.000e-02]]
[[ 3.81957269 93.62131911]
 [ 3.81957269 93.62131911]]


https://github.com/comp-medicine-uc/continuum-lung-mechanics provides a detailed background on the implemented high fidelity model.